In [1]:
import numpy as np
import pandas as pd

/var/folders/gm/c4sdklyj1d5bmsf5thm4k8f00000gn/T/ipykernel_87185/1662815981.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
train_df = pd.read_csv("data/bank-full.csv")
test_df = pd.read_csv("data/bank.csv")
train_df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,yes
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,yes
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,no


In [3]:
X_train = train_df.drop(columns=["y"])
y_train = train_df["y"]

X_test = test_df.drop(columns=["y"])
y_test = test_df["y"]

In [4]:
#from sklearn.tree import DecisionTreeClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

In [16]:
def to_binary(val):
    if val == "yes":
        return 1
    return 0

In [5]:
# Baseline Models
# logrc
# knn
# rfc
# xgbc

In [23]:
pp_obj = ColumnTransformer(
    transformers=[
        ("cat_cols", OneHotEncoder(), ["job", "marital", "education", "default", "housing", "loan", "contact", "day", "month", "poutcome"]),
    ],
    remainder = "passthrough"
)

pl_obj = Pipeline([
    ("preprocessor", pp_obj),
    ("classifier", LogisticRegression())
])

y_train_binary = y_train.apply(to_binary)
pl_obj.fit(X_train, y_train_binary)

y_test_binary = y_test.apply(to_binary)
recall_score(y_test_binary, pl_obj.predict(X_test))

/Users/donaldtaggart/miniconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.2072936660268714

In [24]:
pp_obj = ColumnTransformer(
    transformers=[
        ("cat_cols", OneHotEncoder(), ["job", "marital", "education", "default", "housing", "loan", "contact", "day", "month", "poutcome"]),
    ],
    remainder = "passthrough"
)

pl_obj = Pipeline([
    ("preprocessor", pp_obj),
    ("classifier", KNeighborsClassifier())
])

y_train_binary = y_train.apply(to_binary)
pl_obj.fit(X_train, y_train_binary)

y_test_binary = y_test.apply(to_binary)
recall_score(y_test_binary, pl_obj.predict(X_test))

0.42610364683301344

In [25]:
pp_obj = ColumnTransformer(
    transformers=[
        ("cat_cols", OneHotEncoder(), ["job", "marital", "education", "default", "housing", "loan", "contact", "day", "month", "poutcome"]),
    ],
    remainder = "passthrough"
)

pl_obj = Pipeline([
    ("preprocessor", pp_obj),
    ("classifier", XGBClassifier())
])

y_train_binary = y_train.apply(to_binary)
pl_obj.fit(X_train, y_train_binary)

y_test_binary = y_test.apply(to_binary)
recall_score(y_test_binary, pl_obj.predict(X_test))

0.6852207293666027

In [ ]:
# the above are baselines
# the below are model options

In [26]:
pp_obj = ColumnTransformer(
    transformers=[
        ("cat_cols", OneHotEncoder(), ["job", "marital", "education", "default", "housing", "loan", "contact", "day", "month", "poutcome"]),
        ("quant_cols", StandardScaler(), ["age", "balance", "day", "campaign", "pdays", "previous"])
    ],
    remainder = "passthrough"
)

pl_obj = Pipeline([
    ("preprocessor", pp_obj),
    ("classifier", XGBClassifier())
])

hp_dict = {
    "classifier__max_depth": [12, 14, 16],
    "classifier__subsample": [0.9, 1]
}

gscv_obj = GridSearchCV(pl_obj, hp_dict, cv=5)

def to_binary(val):
    if val == "yes":
        return 1
    return 0

y_train_binary = y_train.apply(to_binary)
gscv_obj.fit(X_train, y_train_binary)

y_test_binary = y_test.apply(to_binary)
recall_score(y_test_binary, gscv_obj.predict(X_test))

1.0

In [27]:
gscv_obj.best_params_

{'classifier__max_depth': 16, 'classifier__subsample': 0.9}